# Merge Datasets and Export To PostgreSQL

## Step-1: Imports Libraries

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import glob
import shutil
from MyCredentials import My_Credentials

## Step-2: Import dataset and merge

In [2]:
# specifying the path to csv files
csv_path = '/Users/pluto/Documents/Code/Test/Used_Cars_Price_Prediction/Scrape_Data/Dataset/CSV'

In [3]:
# csv files in the path
csv_files = glob.glob(csv_path + "/*.csv")

In [4]:
# defining an empty list to store content
content = []

In [5]:
# reading all the csv files in the specified path
for filename in csv_files:
    df = pd.read_csv(filename, index_col=None)
    content.append(df)

In [6]:
# converting content to data frame
data_frame = pd.DataFrame()
data_frame = pd.concat(content)

In [7]:
data_frame.head()

,City,Brand,Model,Body,Certification,Exterior_Color,Interior_Color,Transmission_Type,Fuel_Type,Engine_Type,Wheel_Drive_Type,Miles_Per_Gallon,Year,Zip_Code,Mileage,Price
0,San Antonio,Toyota,Camry,SEDAN,Used,Silver,NaN,Automatic,Gasoline,4-Cylinder,2 wheel drive - front,27 City / 38 Highway,2021,78233,"5,507",38997
1,San Antonio,Toyota,Supra,COUPE,Used,Red,NaN,Automatic,Gasoline,4-Cylinder Turbo,2 wheel drive - rear,25 City / 32 Highway,2021,78233,"7,094",51997
2,San Antonio,Ford,F250,TRUCKS,Used,White,Java,Automatic,Diesel,8-Cylinder Turbo,4 wheel drive,NaN,2020,78219,"19,135",80500
3,San Antonio,Ford,Escape,SUV,Used,Blue,NaN,Automatic,Flexible Fuel,4-Cylinder,2 wheel drive - front,21 City / 29 Highway,2019,78233,"26,196",24997
4,San Antonio,Toyota,RAV4,SUV,Used,Purple,NaN,Automatic,Gasoline,4-Cylinder,2 wheel drive - front,23 City / 30 Highway,2018,78233,"37,486",29287


In [8]:
data_frame.shape

(20000, 16)

## Step-3: Export Dataset to PostgreSQL Database

In [9]:
# copy the dataset
df_merged_used_cars_dataset = data_frame.copy()

### Export to Excel and CSV File

In [10]:
df_merged_used_cars_dataset.to_excel('Raw_Used_Cars_Dataset.xlsx', index=False)

In [11]:
shutil.move('Raw_Used_Cars_Dataset.xlsx', 'Dataset/Merged')

'Dataset/Merged/Raw_Used_Cars_Dataset.xlsx'

In [12]:
df_merged_used_cars_dataset.to_csv('Raw_Used_Cars_Dataset.csv', index=False)

In [13]:
shutil.move('Raw_Used_Cars_Dataset.csv', 'Dataset/Merged')

'Dataset/Merged/Raw_Used_Cars_Dataset.csv'

### Upload file to PostgreSQL table

In [14]:
username = My_Credentials.username
password = My_Credentials.password
hostname = My_Credentials.hostname
port = My_Credentials.port
database = 'Used_Cars_Database'


In [15]:
# create sqlalchemy engine and connect to PostgreSQL server
engine=create_engine(f'postgresql://{username}:{password}@{hostname}:{port}/{database}')

In [16]:
if database_exists(engine.url):
    print('Database exists!')
else:
    create_database(engine.url)
    print('Database created!')

Database exists!


In [17]:
try:
    df_merged_used_cars_dataset.to_sql('Raw_Used_Cars_Dataset', engine, if_exists= 'replace', index= False)
    print('Data uploaded to PostgreSQL table')

except:
    print("Sorry, some error has occurred!")

finally:
    engine.dispose()

Data uploaded to PostgreSQL table
